In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# import only frequently used modules
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Adult Dataset

##### **[Adult-1]** Dataset Preparation

In [4]:
adult = pd.read_csv("./data/adult.csv")
adult = adult.drop(columns=["fnlwgt"])
adult = adult.replace({"?": np.nan})
adult = adult.dropna()
adult = adult.replace({"<=50K": 0, ">50K": 1})
adult = adult.reset_index(drop=True)

In [6]:
from sklearn.model_selection import train_test_split

X = adult.drop(columns=["income"])
y = adult["income"]

X_train_, X_test_, y_train_, y_test_ = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
# transform categorical features to one-hot encoding, and scale numerical features
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_features = X.select_dtypes(include=["object", "bool"]).columns
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

numerical_features = X.select_dtypes(include=["int64", "float64"]).columns
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numerical_transformer, numerical_features),
    ]
)

X_train: np.ndarray = preprocessor.fit_transform(X_train_).A  # type: ignore
X_test: np.ndarray = preprocessor.transform(X_test_).A  # type: ignore

y_train: np.ndarray = y_train_.to_numpy()  # type: ignore
y_test: np.ndarray = y_test_.to_numpy()  # type: ignore

✅ now `X_train`, `X_test`, `y_train`, `y_test` are prepared

##### **[Adult-2]** Experiment

In [8]:
from utils import get_params_combination

adult_params = get_params_combination({
    "alpha": [0.0],
    "gamma": list(np.linspace(0.001, 0.4, 20)),
    "r": [5.0],
    "nu": [0.04],
    "lambda_max": [10.0],
})

pd.DataFrame(adult_params)  # wrapping with pd.DataFrame just for tabular display

,alpha,gamma,r,nu,lambda_max
0,0.0,0.001,5.0,0.04,10.0
1,0.0,0.022,5.0,0.04,10.0
2,0.0,0.043,5.0,0.04,10.0
3,0.0,0.064,5.0,0.04,10.0
4,0.0,0.085,5.0,0.04,10.0
5,0.0,0.106,5.0,0.04,10.0
6,0.0,0.127,5.0,0.04,10.0
7,0.0,0.148,5.0,0.04,10.0
8,0.0,0.169,5.0,0.04,10.0
9,0.0,0.190,5.0,0.04,10.0


In [9]:
from experiments import MainExperiment
from tasks import BinaryLogisticClassificationTask

adult_exp = MainExperiment(BinaryLogisticClassificationTask)

adult_exp.task.train(X_train, y_train)
adult_exp.task.test(X_test, y_test)

adult_results = adult_exp.run(adult_params)

5 / 20 [1] Generating oracle caches: 100%|██████████| 200/200 [02:35<00:00,  1.29it/s]]
5 / 20 [2] Generating lagrangian caches: 200it [00:00, 3882.77it/s]


5 / 20 [3] Solving:   0%|          | 0/60416 [00:00<?, ?it/s]

11 / 20 [1] Generating oracle caches: 100%|██████████| 200/200 [02:35<00:00,  1.29it/s]
9 / 20 [2] Generating lagrangian caches: 200it [00:00, 4023.24it/s]


9 / 20 [3] Solving:   0%|          | 0/78828 [00:00<?, ?it/s]

10 / 20 [1] Generating oracle caches: 100%|██████████| 200/200 [02:35<00:00,  1.29it/s]
10 / 20 [2] Generating lagrangian caches: 0it [00:00, ?it/s]

...

5 / 20 [3] Solving:  60%|██████    | 36337/60416 [00:00<00:00, 60732.19it/s]

15 / 20 [2] Generating lagrangian caches: 200it [00:00, 1264.60it/s]


15 / 20 [3] Solving:   0%|          | 0/111032 [00:00<?, ?it/s]

14 / 20 [2] Generating lagrangian caches: 200it [00:00, 1128.43it/s]
6 / 20 [2] Generating lagrangian caches: 200it [00:00, 1174.53it/s]



19 / 20 [3] Solving: 100%|██████████| 135558/135558 [00:02<00:00, 60581.53it/s]


In [10]:
target_alpha = 0.0
target_r = 5.0

plot_x = []
plot_I_alpha = []
plot_err = []

for param in adult_results:
    param_dict = {k: v for k, v in list(param)}

    if param_dict["alpha"] != target_alpha or param_dict["r"] != target_r:
        continue

    plot_x.append(param_dict["gamma"])
    plot_I_alpha.append(adult_results[param][2])
    plot_err.append(adult_results[param][3])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(plot_x, plot_I_alpha)
plt.xlabel("gamma")
plt.ylabel("I(alpha)")

In [ ]:
plt.plot(plot_x, plot_err)
plt.xlabel("gamma")
plt.ylabel("err")